# Basic Example: Prompt+Model+OutputParser



## Overview

最基本最常見的用法是將提示模板與模型連結起來。為了說明如何使用，我們將建立一個鏈，要求提供各種國家的首都。


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Utilizing Prompt Templates](#utilizing-prompt-templates)
- [Chain Creation](#chain-creation)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Python List Tutorial](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "<Your OpenAI API KEY>",
        "LANGCHAIN_API_KEY": "<Your LangChain API KEY>",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "",
    }
)

Environment variables have been set successfully.


---
你也可以在 ```.env``` 文件中設定 ```OPENAI_API_KEY```，並且載入它。 

[Note] 如果你已經在之前的步驟中设置了 ```OPENAI_API_KEY```，那麽這一步是不必要的。

In [3]:
# Configuration File for Managing API Key as an Environment Variable
from dotenv import load_dotenv

# Load API KEY Information
load_dotenv(override=True)

True

In [4]:
# Set up LangSmith tracking: https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

False

## Utilizing Prompt Templates

```PromptTemplate```

- PromptTemplate常用來 創造一個完整的提示字串，其中包含使用者的輸入變數。
- Usage
  - ```template```: 模板字串是一個預先定義的格式，其中使用大括號 '{}' 來表示變數。

  - ```input_variables```: 定義了要插入大括號中的變數名稱的清單。

```input_variables```

- ```input_variables```定義了要插入大括號中的變數名稱的清單。

In [5]:
from langchain_core.prompts import PromptTemplate

The ```from_template()``` method is used to create a ```PromptTemplate``` object.

In [6]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [7]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [8]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [9]:
# Generate the prompt.
prompt = prompt_template.format(country="USA")
prompt

'What is the capital of USA?'

In [9]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

## Chain Creation

### LCEL (LangChain Expression Language)

這裏, 我們使用 LCEL 將各種組件組合成一個單一的鏈.

![lcel.png](./assets/02-langchain-expression-language.png)

```
chain = prompt | model | output_parser
```

```|``` 符號類似於 [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>), 連結不同的組件並傳遞一個組件的輸出作為下一個組件的輸入.

在這個鏈中, 使用者輸入被傳遞到提示模板, 並從提示模板輸出轉發到模型. 通過獨立檢查每個組件, 你可以理解每個步驟發生什麼.

In [10]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


# Combine the prompt and model into a chain
chain = prompt | model

### Calling ```invoke()```

- 輸入值以 Python 字典 (key-value pairs) 的形式提供.  
- 在呼叫 ```invoke()``` 函數時, 這些輸入值會作為參數傳遞.

In [11]:
# 設定   topic 在 `input` 字典中為 'The Principles of Learning in Artificial Intelligence Models'.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [12]:
# 將 `prompt` 物件和 `model` 物件使用管道 (`|`) 運算子連接.
# 使用 `invoke` 方法傳遞 `input`.
# 這將返回 AI 模型生成的消息.
chain.invoke(input)

AIMessage(content="Sure! The principles of learning in artificial intelligence (AI) models can be understood through a few key concepts. Here’s a simple breakdown:\n\n1. **Data**: AI models learn from data. Just like humans learn from experiences, AI systems analyze large amounts of information (data) to understand patterns and make decisions.\n\n2. **Training**: This is the process where the AI model learns from the data. During training, the model is exposed to examples and adjusts its internal parameters to improve its performance. Think of it like practicing a sport; the more you practice, the better you get.\n\n3. **Feedback**: After training, the model is tested on new data to see how well it performs. Feedback is crucial because it helps the model understand what it did right and what it needs to improve. This is similar to getting grades on a test.\n\n4. **Generalization**: A good AI model should not just memorize the training data but should also be able to apply what it learn

下面是一個輸出流式回應的示例:

In [13]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Sure! The principles of learning in artificial intelligence (AI) models can be understood through a few key concepts. Here’s a simple breakdown:

1. **Data**: AI models learn from data. Just like humans learn from experiences, AI systems analyze large amounts of data to find patterns and make decisions. The quality and quantity of data are crucial for effective learning.

2. **Training**: This is the process where the AI model learns from the data. During training, the model adjusts its internal parameters to minimize errors in its predictions. Think of it like practicing a sport; the more you practice, the better you get.

3. **Feedback**: AI models often use feedback to improve. After making predictions, they compare their results to the actual outcomes. This feedback helps the model understand what it did right or wrong, allowing it to adjust and improve over time.

4. **Generalization**: A good AI model should not only perform well on the data it was trained on but also on new, uns

### Output Parser

一個 **Output Parser** 是一個工具, 用於將 AI 模型的回應轉換或處理成特定格式. 由於模型的輸出通常以自由格式文本提供, 一個 **Output Parser** 是必要的, 用於將其轉換為結構化格式或提取所需數據.


In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = (
    StrOutputParser()
)  # 直接返回模型的回應作為字符串, 不進行修改.

An output parser is added to the chain.

In [ ]:
# A processing chain is constructed by connecting the prompt, model, and output parser.
chain = prompt | model | output_parser 

In [16]:
# Use the invoke method of the chain object to pass the input
chain.invoke(input)

'Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more relevant and high-quality data you provide, the better the model can learn. Think of it like teaching a child; the more books and experiences they have, the more they learn.\n\n2. **Patterns and Features**: AI looks for patterns in the data. It identifies features (important characteristics) that help it understand and make predictions. For example, if you\'re teaching an AI to recognize cats in pictures, it will learn features like fur texture, ear shape, and eye color.\n\n3. **Learning from Examples**: AI models learn by example. You show them many examples of what you want them to learn (like pictures of cats and dogs), and they adjust their understanding based on these examples.\n\n4. **Feedback and Improvement**: Ju

In [17]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:

1. **Data**: AI models learn from data. The more relevant and high-quality data you provide, the better the model can learn. Think of data as the "food" that helps the model grow and improve.

2. **Patterns**: AI models look for patterns in the data. Just like humans recognize faces or voices, AI tries to find similarities and trends in the information it receives. This helps the model make predictions or decisions.

3. **Feedback**: Learning often involves feedback. When an AI model makes a mistake, it can learn from that mistake if it receives feedback. This is similar to how we learn from our errors and adjust our behavior accordingly.

4. **Generalization**: A good AI model should not just memorize the data it was trained on; it should be able to apply what it learned to new

### Applying and Modifying Templates

- The prompt content below can be **modified** as needed for testing purposes.  
- The ```model_name``` can also be adjusted for testing.

In [20]:
template = """
你是有 10 年經驗的英文老師, 請寫一個適合給定情況的英文對話.  
請參考 [FORMAT] 的結構.

# 情況:
{question}

# 格式:
- 英文對話:
- 英文對話的解釋: 
"""

# 產生 prompt
prompt = PromptTemplate.from_template(template)

# 初始化 ChatOpenAI 模型
model = ChatOpenAI(model_name="gpt-4o-mini")

# 初始化 string output parser
output_parser = StrOutputParser()

In [21]:
# Construct the chain.
chain = prompt | model | output_parser

In [22]:
# Execute the completed Chain to obtain a response.
print(chain.invoke({"question": "我想要去餐廳訂餐."}))

- 英文對話:
A: Good evening! Thank you for calling Bella's Italian Restaurant. How can I help you today?  
B: Hi! I would like to make a reservation for dinner.  
A: Sure! For what date and time?  
B: I’d like to reserve a table for two this Friday at 7 PM.  
A: Great! Can I have your name, please?  
B: Yes, it’s Sarah.  
A: Thank you, Sarah. Your reservation is confirmed for two people this Friday at 7 PM. Is there anything else I can assist you with?  
B: No, that’s all. Thank you!  
A: You’re welcome! We look forward to seeing you on Friday. Have a great day!

- 英文對話的解釋:
這段對話中，A是餐廳的接待員，而B是想要預訂餐桌的顧客。B首先表達了想要預訂的意圖，然後A詢問具體的日期和時間。B提供了預訂的詳細信息，包括人數和名字。A確認了預訂並詢問是否需要其他幫助，最後B表示不需要更多的幫助，並感謝A。這是一個典型的餐廳預訂的對話範例。


In [23]:
# Execute the completed Chain to obtain a response
# Request for Streaming Output
answer = chain.stream({"question": "我想要去餐廳訂餐."})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- 英文對話:  
**Customer:** Hi, I'd like to make a reservation for dinner, please.  
**Host:** Sure! For how many people?  
**Customer:** It will be for four people.  
**Host:** Great! What date and time are you looking at?  
**Customer:** We would like to come on Saturday at 7 PM.  
**Host:** Perfect! I have you down for four people on Saturday at 7 PM. Can I have your name, please?  
**Customer:** Yes, it's Alex.  
**Host:** Thank you, Alex! We look forward to seeing you then.  

- 英文對話的解釋:  
這段對話是一位顧客與餐廳接待員之間的對話。顧客首先表達了想要訂位的要求，接待員隨後詢問訂位的人數。顧客回答是四個人，然後接待員詢問具體的日期和時間。在顧客提供了他們想要的時間後，接待員確認了預訂並要求顧客提供名字。最後，顧客提供了名字，接待員感謝並表示期待顧客的到來。這段對話簡單且清晰，適合用於餐廳訂位的情境。

In [24]:
# This time, set the question to 'Ordering Pizza in the US' and execute it.
# Request for Streaming Output
answer = chain.stream({"question": "在美國訂購比薩"})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- 英文對話:
**Customer:** Hi, I’d like to order a pizza, please.  
**Pizza Shop:** Sure! What size would you like?  
**Customer:** I’ll have a large pizza.  
**Pizza Shop:** Great! What toppings would you like?  
**Customer:** Can I get pepperoni, mushrooms, and extra cheese?  
**Pizza Shop:** Absolutely! Would you like to add any drinks or sides?  
**Customer:** Yes, I’ll take a two-liter soda and some garlic bread.  
**Pizza Shop:** Perfect! Your total comes to $25. Would you like to pay with cash or card?  
**Customer:** I’ll pay with my card.  
**Pizza Shop:** Thank you! Your order will be ready in about 30 minutes.  
**Customer:** Thank you!

- 英文對話的解釋: 
這段對話展示了顧客在美國的比薩店訂購比薩的過程。顧客開始時詢問訂購，店員確認訂單的大小。接著，顧客選擇了比薩的配料，店員則詢問是否需要附加的飲料或配菜，顧客選擇了飲料和大蒜麵包。最後，店員告知總金額並詢問付款方式，顧客選擇了使用信用卡付款，並確認了預計取餐的時間。這段對話簡單明了，符合在美國訂購比薩的實際情境。